# Modal Analysis of a Bernoulli Console

A short comparison of different modeling techniques of a simple console. The beam has a rectangular prismatic cross-section and a linear elastic material model, governed by the following parameters:

In [1]:
# all units in kN and cm
L = 150.  # length of the console [cm]
F = 1.  # value of the vertical load at the free end [kN]
E = 21000.0  # Young's modulus [kN/cm3]
nu = 0.3  # Poisson's ratio [-]
w, h = 5., 15.  # width and height of the rectangular cross section
g = 9.81  # gravitational acceleration [m/s2]
density = 7750.0 * 1e-6 # mass density [g/cm3]
nElem = 20  # number of subdivisons to use

In [2]:
A = w * h  # area
Iy = w * h**3 / 12  # second moment of inertia around the y axis
Iz = h * w**3 / 12  # second moment of inertia around the z axis
Ix = Iy + Iz  # torsional inertia
weight = density * 1e-3 * g  # [kN/cm3]
dpa = density * A  # density per area [g/cm]
wpa = weight * A  # weight per area [kN/cm]
mass = L * dpa  # total mass

In [3]:
from sigmaepsilon.solid import Structure, LineMesh, PointData
from neumann.linalg import linspace, Vector
from polymesh.space import StandardFrame, PointCloud, frames_of_lines
from sigmaepsilon.solid.fem.cells import B2 as Beam
import numpy as np

# model stiffness matrix
G = E / (2 * (1 + nu))
Hooke = np.array([
    [E*A, 0, 0, 0],
    [0, G*Ix, 0, 0],
    [0, 0, E*Iy, 0],
    [0, 0, 0, E*Iz]
])

# space
GlobalFrame = StandardFrame(dim=3)

# mesh
p0 = np.array([0., 0., 0.])
p1 = np.array([L, 0., 0.])
coords = linspace(p0, p1, nElem+1)
coords = PointCloud(coords, frame=GlobalFrame).show()
topo = np.zeros((nElem, 2), dtype=int)
topo[:, 0] = np.arange(nElem)
topo[:, 1] = np.arange(nElem) + 1

# load at the rightmost node in Y and Z directions
nodal_loads = np.zeros((coords.shape[0], 6, 3))
global_load_vector = Vector([F,  0, 0.], frame=GlobalFrame).show()
nodal_loads[-1, :3, 0] = global_load_vector
global_load_vector = Vector([0., F, 0.], frame=GlobalFrame).show()
nodal_loads[-1, :3, 1] = global_load_vector
global_load_vector = Vector([0., 0, F], frame=GlobalFrame).show()
nodal_loads[-1, :3, 2] = global_load_vector

# support at the leftmost node (all degrees)
fixity = np.zeros((coords.shape[0], 6)).astype(bool)
fixity[0, :] = True

# mass and density
nodal_masses = np.zeros((coords.shape[0],))
densities = np.full((topo.shape[0],), dpa)

# pointdata
pd = PointData(coords=coords, loads=nodal_loads, 
               fixity=fixity, mass=nodal_masses)

# celldata
frames = frames_of_lines(coords, topo)
cd = Beam(topo=topo, frames=frames, density=densities)

# set up mesh and structure
mesh = LineMesh(pd, cd, model=Hooke, frame=GlobalFrame)
structure = Structure(mesh=mesh)

# perform linear analysis
structure.linsolve()

# calculate the mass matrix
structure.consistent_mass_matrix()

# postproc
dofsol = structure.nodal_dof_solution()

## Dunkerley's Approximation

Lower bound estimations on the smallest natural circular frequencies by *splitting the masses*.

In [4]:
def Dunkerley(N:int=1):
    mi = mass / N
    h = L / N
    i1 = np.sum(list(range(1, N + 1)))
    i3 = np.sum([i**3 for i in range(1, N + 1)])
    acc_x = i1 * mi * h / (E * A)
    acc_y = i3 * mi * h**3 / (3 * E * Iy)
    acc_z = i3 * mi * h**3 / (3 * E * Iz)
    return np.sqrt(1/acc_x), np.sqrt(1/acc_y), np.sqrt(1/acc_z)


Dunkerley(20)

(15.14564891325506, 1.0451490688928273, 0.3483830229642758)

## Rayleigh's Approximtion

Upper bound estimations of the smallest natural circular frequencies using the equivalence of elastic internal energy and kinetic energy of undamped conservative systems.

In [5]:
from sigmaepsilon.solid.fem.dyn import Rayleigh_quotient


def Rayleigh(structure):
    M = structure.Solver.M
    u = structure.Solver.u
    f = structure.Solver.f
    return np.sqrt(Rayleigh_quotient(M, u=u, f=f))


Rayleigh(structure)

array([19.00763833,  1.13004384,  0.37668141])

## FEM Approximation

By solving the eigenvalue problem using dense matrices:

In [6]:
structure.consistent_mass_matrix()
freks, modes = structure.natural_circular_frequencies(normalize=True, as_dense=True, 
                                                      return_vectors=True)
freks[:5]

array([0.37124576, 1.11373705, 2.32512818, 6.50404484, 6.97538265])

In [7]:
around = min(Rayleigh(structure))
freks, modes = structure.natural_circular_frequencies(k=40, return_vectors=True, 
                                                      which='SM')
freks[np.where(np.abs(freks) < 1e-3)] = 0.
freks[:5]

array([0.37124581, 1.11373705, 2.32512837, 6.50404513, 6.97538265])

Using Rayleigh's approximation to help the sparse solver:

In [8]:
around = min(Rayleigh(structure))**2
freks, modes = structure.natural_circular_frequencies(k=20, around=around, 
                                                      return_vectors=True, which='LM')
freks[np.where(np.abs(freks) < 1e-3)] = 0.
freks[:5]

array([0.37124581, 1.11373705, 2.32512836, 6.50404513, 6.97538265])

## Effective Modal Masses

In [9]:
from sigmaepsilon.solid.fem.dyn import effective_modal_masses

nN, nD, nR = dofsol.shape

action_x = np.zeros((nN, nD))
action_x[:, 0] = 1.0
action_x = action_x.reshape(nN * nD)
action_y = np.zeros((nN, nD))
action_y[:, 1] = 1.0
action_y = action_y.reshape(nN * nD)
action_z = np.zeros((nN, nD))
action_z[:, 2] = 1.0
action_z = action_z.reshape(nN * nD)
actions = np.stack([action_x, action_y, action_z], axis=1)

M = structure.Solver.M
m_eff = effective_modal_masses(M, actions, modes)

m_eff[np.where(m_eff <= 1e-5)] = 0.
for i in range(10):
    print(m_eff[i], freks[i])

[ 0.          0.         53.44103997] 0.3712458129958959
[ 0.         53.44104933  0.        ] 1.113737049154143
[ 0.          0.         16.41110569] 2.3251283643995957
[0.         0.         5.63602212] 6.504045132114754
[ 0.         16.41110859  0.        ] 6.975382651082683
[0.         0.         2.87706282] 12.72743946381008
[70.67150017  0.          0.        ] 17.242452035296814
[0.         5.63602315 0.        ] 19.512128559977167
[0.         0.         1.73728562] 21.002115632827188
[0.         0.         1.16046247] 31.307695276172957


The sum of effective masses for an action converges to the total mass of the structure as the mesh is refined, but since some masses get distributed to fixed supports, it will never reach the total mass. 

In [10]:
mass_fem = np.sum(m_eff[:, 0]), np.sum(m_eff[:, 1]), np.sum(m_eff[:, 2])
mass, mass_fem

(87.1875, (81.3498524777421, 81.2629932099404, 83.57762006238761))

The percetage of the sums of effective masses to the total mass:

In [11]:
["{:.2f}%".format(100 * m / mass) for m in mass_fem]

['93.30%', '93.20%', '95.86%']

## Bonus

In the previous FEM approximation we represented the mass by defining density values for the cells. Although we always have to define a density distribution for the cells, this can be augmented by nodal masses. The following block distributes the masses to the nodes, and cell-densities are used to guarantee that the diagonal of the mass matrix is always filled up.

Note that this calculation is inferior to the variational approach, since rotational masses are neglected, as all the mass is delegated to translational DOFs.

In [12]:
# mass and density
nN = coords.shape[0]  # the number of nodes in the model
min_cell_mass = mass / 1000
cell_density = min_cell_mass / L
densities = np.full((topo.shape[0],), cell_density)
mass_on_nodes = (mass - min_cell_mass) / nN
nodal_masses = np.full((coords.shape[0],), mass_on_nodes)

# pointdata
pd = PointData(coords=coords, frame=GlobalFrame, loads=nodal_loads, 
               fixity=fixity, mass=nodal_masses)

# celldata
frames = frames_of_lines(coords, topo)
cd = Beam(topo=topo, frames=frames, density=densities)

# set up mesh and structure
mesh = LineMesh(pd, cd, model=Hooke, frame=GlobalFrame)
structure = Structure(mesh=mesh)
structure.linsolve()

# postproc
dofsol = structure.nodal_dof_solution()

In [13]:
structure.consistent_mass_matrix()
freks, modes = structure.natural_circular_frequencies(normalize=True, as_dense=True, 
                                                      return_vectors=True)
M = structure.Solver.M
u = structure.Solver.u
f = structure.Solver.f
f = f.reshape(u.shape)

nN, nD, nR = dofsol.shape

action_x = np.zeros((nN, nD))
action_x[:, 0] = 1.0
action_x = action_x.reshape(nN * nD)
action_y = np.zeros((nN, nD))
action_y[:, 1] = 1.0
action_y = action_y.reshape(nN * nD)
action_z = np.zeros((nN, nD))
action_z[:, 2] = 1.0
action_z = action_z.reshape(nN * nD)
actions = np.stack([action_x, action_y, action_z], axis=1)

m_eff = effective_modal_masses(M, actions, modes)
mtot = np.sum(m_eff[:, 0]), np.sum(m_eff[:, 1]), np.sum(m_eff[:, 2])
L * dpa, mtot

m_eff[np.where(m_eff <= 1e-5)] = 0.
for i in range(10):
    print(m_eff[i], freks[i])

[ 0.         11.20967175 40.29152448] 0.5979959737488234
[ 0.         40.99848629 11.2436472 ] 0.9777954186084489
[0.00000000e+00 2.91389119e-03 1.66822764e+01] 2.2735307177352477
[0.         0.         5.52062605] 6.374141992513992
[ 0.         16.01825595  0.        ] 6.821159228051858
[0.         0.         2.81608947] 12.505791384469797
[68.92295576  0.          0.        ] 17.22876308195084
[0.         5.50934657 0.        ] 19.122414488855828
[0.         0.         1.70239872] 20.696305910850615
[0.         0.         1.13869864] 30.947495975147174


In [14]:
mass_fem = np.sum(m_eff[:, 0]), np.sum(m_eff[:, 1]), np.sum(m_eff[:, 2])
["{:.2f}%".format(100 * m / mass) for m in mass_fem]

['100.00%', '100.00%', '100.00%']

Check Rayleigh's approximation with the new mass matrix:

In [15]:
Rayleigh(structure)

array([18.77463591,  1.09986931,  0.36662323])

In [ ]:
# Import required module
import texttable

# Create texttable object
tableObj = texttable.Texttable()

# Set columns
tableObj.set_cols_align(["l", "r", "c"])

# Set datatype of each column
tableObj.set_cols_dtype(["a", "i", "t"])

# Adjust columns
tableObj.set_cols_valign(["t", "m", "b"])

# Insert rows
tableObj.add_rows([
		["ORGANIZATION", "ESTABLISHED", "CEO"],
		["Google", 1998, "Sundar Pichai"],
		["Microsoft", 1975, "Satya Nadella"],
		["Nokia", 1865, "Rajeev Suri"],
		["Geeks for Geeks", 2008, "Sandeep Jain"],
		[None, 2007, "Vivek Ravisankar"]
		])

# Display table
print(tableObj.draw())